D'abord, installation de Spark


In [ ]:
import os
memory = '8g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [ ]:
!gsutil -m cp -r gs://angelo_crime_data/spark-3.2.1-bin-hadoop2.7.tgz .

In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz

In [ ]:
!pip install findspark pyspark

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "./spark-3.2.1-bin-hadoop2.7"


findspark.init()

# First ML model using Apache Spark MLlib


## Load the data

To load the data we are using Spark DataFrames. Spark it’s a little bit more complicated than Pandas. You can’t just do “import -> read_csv()”. You first need to start a Spark Session, to do that write:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('Titanic Data') \
    .getOrCreate()

In [ ]:
spark

Cool! So now we have everything in place to read the data. To do so write:

In [ ]:
!wget https://raw.githubusercontent.com/lsteffenel/RT0902-IntroML/main/data/titanic_train.csv

In [ ]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("titanic_train.csv"))

And that’s it! You have created your TITANIC Spark DataFrame. To see the internals of the DataFrame write:

In [ ]:
df.show(5)

One good thing about using Python is that you can interact with Pandas easily. And to show our data in a prettier format you can write:

In [ ]:
df.toPandas()

## Checking information about your data

In [ ]:
# How many rows we have
df.count()

In [ ]:
# The names of our columns
df.columns

In [ ]:
# Basics stats from our columns
df.describe().toPandas()

In [ ]:
# Types of our columns
df.dtypes

## Data preparation and feature engineering

One of the things we noticed from the data exploration from above was that all the columns were of String type. But that doesn’t seem right. Some of them should be numeric. So we are going to cast them. Also because of time I’m only selecting a few variables for modeling so we don’t have to deal with the whole dataset:

In [ ]:
# Cast numeric columns

from pyspark.sql.functions import col

dataset = df.select(col('Survived').cast('float'),
                         col('Pclass').cast('float'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('Fare').cast('float'),
                         col('Embarked')
                        )

dataset.show()

In [ ]:
## See if we have missing values
from pyspark.sql.functions import isnull, when, count, col

dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

We see that we also have null values in some columns, so we will just eliminate them:

In [ ]:
# Drop missing values
dataset = dataset.replace('null', None)\
        .dropna(how='any')

Now, the Spark ML library only works with numeric data. But we still want to use the Sex and the Embarked column. For that, we will need to encode them. To do it let’s use something called the StringIndexer:

In [ ]:
# Index categorical columns with StringIndexer
from pyspark.ml.feature import StringIndexer
dataset = StringIndexer(
    inputCol='Sex', 
    outputCol='Gender', 
    handleInvalid='keep').fit(dataset).transform(dataset)
dataset = StringIndexer(
    inputCol='Embarked', 
    outputCol='Boarded', 
    handleInvalid='keep').fit(dataset).transform(dataset)
dataset.show()

As you can see we’ve created two new columns “Gender” and “Boarded” that contain the same information as “Sex” and “Embarked” but now they are numeric. Let’s do a final check for our data types:

In [ ]:
# Check data types
dataset.dtypes

So all the columns we want are numeric. We now have to get rid of the old columns “Sex” and “Embarked” because we won’t be using them:

In [ ]:
# Drop unnecessary columns
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

dataset.show()

Jut one step left before going into the machine learning part. Spark actually works to predict with a column with all the features smashed together into a list-like structure. 

But you want to predict “Survived”, so you need to combine the information of the columns “Pclass”, “Age”, “Fare”, “Gender” and “Boarded” into one column. To do that in Spark we use the VectorAssembler:

In [ ]:
# Assemble all the features with VectorAssembler

required_features = ['Pclass',
                    'Age',
                    'Fare',
                    'Gender',
                    'Boarded'
                   ]

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=required_features, outputCol='features')

transformed_data = assembler.transform(dataset)

In [ ]:
transformed_data.show()

## Modeling

Now for the fun part right? NO! Haha. Modeling is important but without all the previous steps it would be impossible. So have fun in all the steps :)
Before modeling let’s do the usual splitting between training and testing:

In [ ]:
# Split the data
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

Ok. Modeling. That means, in this case, build and fit an ML model to our dataset to predict the “Survived” columns with all the other ones. We will be using a Random Forest Classifier. This is actually an estimator that we have to fit.
This is actually the easy part:

In [ ]:
# Define the model
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='Survived', 
                            featuresCol='features',
                            maxDepth=5)

Now we fit the model:

In [ ]:
# Fit the model
model = rf.fit(training_data)

This will give us something called a transformer. And finally, we predict using the test dataset:

In [ ]:
# Predict with the test dataset
predictions = model.transform(test_data)

And that’s it! You did it. Congratulations :). Your first Spark ML model. 

Now let’s see how well we did. For that, we will use a basic metric called the accuracy:

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived', 
    predictionCol='prediction', 
    metricName='accuracy')

In [ ]:
# Accuracy
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy = ', accuracy)